### Назначение ноутбука

В этом ноутбуке реализовано обучение и оценка качества моделей рекомендаций на основе различных подходов:

1. **Baseline 0** — рекомендации по популярности (простая эвристика).
2. **Baseline 1** — модель CatBoost с текстовыми TF-IDF признаками.
3. **MatchingMLP** — нейросетевая модель, обучающаяся на конкатенированных векторах пользователя и товара.

Данные использованы после предварительной разметки данных на основе поставленной пользователем оценки товару, а именно 1 при оценке >= 4, и 0 - в иных случаях (rating-based sampling).


In [4]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys
import os
import random
import torch

disable_tqdm = os.getenv("TQDM_DISABLE", "0") == "1"

In [5]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

### Импорт данных и предобработка

Загружаются:

* `df_train` — обучающая выборка с метками взаимодействия;
* `df_test` — лог реальных взаимодействий пользователей;
* `df_meta` — признаки товаров, включая эмбеддинги CLIP и TF-IDF, числовые и категориальные признаки.

Для использования в разных моделях далее будут созданы отдельные версии датафрейма:

* `df_meta_tfidf` — без CLIP-признаков (только TF-IDF);
* `df_meta_clip` — без TF-IDF (только CLIP + табличные признаки).


In [6]:
# Загружаем тестовый датасет и обучающую выборку (rating-based sampling)
df_test = pd.read_csv("data/df_test_ground_truth_rating_based.csv")
df_train = pd.read_csv(
    "data/df_train_baseline_0_rating_based.csv",
    na_values=[""],  # исключаем "Unknown"
    keep_default_na=False
)

# Загружаем мета-информацию с признаками TF-IDF и CLIP
df_meta = pd.read_csv(
    "data/amazon_meta_clean.csv",
    na_values=[""],  # исключаем "Unknown"
    keep_default_na=False
)
# Приводим название колонки к общему формату
df_meta.rename(columns={"asin": "item_id"}, inplace=True)
# Удаляем исходные текстовые и визуальные поля, т.к. они не используются напрямую
df_meta.drop(columns=["text_full", "image_main"], inplace=True)

In [7]:
# Для TF-IDF модели: удалим CLIP-фичи
tfidf_cols = [col for col in df_meta.columns if col.startswith("clip_text_") or col.startswith("clip_img_")]
df_meta_tfidf = df_meta.drop(columns=tfidf_cols)

In [8]:
# Для CLIP модели: удалим TF-IDF-фичи
clip_cols = [col for col in df_meta.columns if col.startswith("tfidf_")]
df_meta_clip = df_meta.drop(columns=clip_cols)

## Baseline 0: Popularity-Based Recommender

Наиболее простая стратегия: каждому пользователю рекомендуются самые популярные товары (наиболее часто встречающиеся среди положительных взаимодействий в трейне).

### Преимущества

- Очень быстро считается
- Не требует персонализации или фичей
- Может служить нижней границей качества модели

### Ограничения

- Не учитывает интересы конкретного пользователя
- Невозможно адаптироваться под редкие/новые товары
- Игнорирует временную динамику и мультимодальные признаки

---

Мы сравним этот бейзлайн с более продвинутыми моделями позже: CatBoost, CLIP + текст/изображения и т.д.


In [9]:
# Самые популярные товары (по кол-ву позитивных взаимодействий)
popular_items = (
    df_train[df_train["label"] == 1]["item_id"]
    .value_counts()
    .head(10)
    .index
    .tolist()
)

In [10]:
# Предсказание: каждому пользователю - один и тот же топ-10
predictions = {user: popular_items for user in df_test["user_id"].unique()}

In [11]:
# Ground truth
# Сопоставляем каждому пользователю множество товаров,
# с которыми он взаимодействовал в тесте
ground_truth = df_test.groupby("user_id")["item_id"].apply(set).to_dict()

## Оценка качества: Precision@K и MAP@K

После генерации рекомендаций для каждого пользователя (`predictions`) мы хотим оценить качество модели с помощью стандартных метрик:

### Precision@K
- Показывает долю релевантных товаров (из ground truth), попавших в top-K рекомендаций
- Рассчитывается для каждого пользователя, затем усредняется

### MAP@K (Mean Average Precision)
- Учитывает не только попадание, но и **позицию** релевантного товара в списке
- Чем выше релевантные товары в списке — тем лучше



In [12]:
# Функция Precision@K — средняя доля релевантных товаров среди top-K рекомендаций
def precision_at_k(preds, ground_truth, k=10):
    scores = []
    for user, pred_items in preds.items():
        if user not in ground_truth:
            continue
        gt_items = ground_truth[user]
        hits = sum([1 for item in pred_items[:k] if item in gt_items])
        scores.append(hits / k)
    return round(np.mean(scores), 4)

In [13]:
# Функция average precision для одного пользователя
def apk(pred, actual, k=10):
    if not actual:
        return 0.0
    pred = pred[:k]
    score, num_hits = 0.0, 0.0
    for i, p in enumerate(pred):
        if p in actual and p not in pred[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(actual), k)

In [14]:
# Средняя average precision по всем пользователям
def map_at_k(preds, ground_truth, k=10):
    return round(
        np.mean([
            apk(preds[u], ground_truth[u], k)
            for u in preds if u in ground_truth
        ]),
        4
    )

In [15]:
# Оцениваем качество рекомендаций по популярности (Baseline 0)
print("Precision@10:", precision_at_k(predictions, ground_truth, k=10))
print("MAP@10:", map_at_k(predictions, ground_truth, k=10))

Precision@10: 0.0015
MAP@10: 0.002


## Baseline 1: TF-IDF + PCA + CatBoost

Следующий бейзлайн использует простую, но более информативную стратегию: **машинное обучение на основе текстовых признаков**.

### Идея

- Используем текстовое описание товара (`text_full`) и превращаем его в числовой вектор с помощью **TF-IDF**.
- Чтобы избежать переобучения и снизить размерность — применяем **PCA**.
- Модель **CatBoost** обучается на этих эмбеддингах, предсказывая вероятность релевантности товара для пользователя.

### Преимущества

- Учитывает смысловое содержание описания товара.
- Обеспечивает некоторую степень персонализации.
- Прост в реализации и достаточно эффективен на структурированных данных.

### Ограничения

- Не использует изображение (визуальную составляющую товара).
- Не моделирует поведение пользователя напрямую (работает на item-фичах).
- Возможны потери информации при снижении размерности через PCA.

---

Этот бейзлайн служит первым шагом от простых эвристик к ML-подходам. Далее мы добавим **визуальные эмбеддинги CLIP**, чтобы построить мультимодальную модель.


In [12]:
from catboost import CatBoostClassifier, Pool

In [13]:
# Загружаем датасет для baseline-модели на TF-IDF фичах
df_train = pd.read_csv(
    "data/df_train_baseline_1_rating_based.csv",
    na_values=[""],  # исключаем "Unknown"
    keep_default_na=False
)

In [14]:
# Списки признаков: tf-idf фичи, числовые признаки, категориальные признаки
tfidf_cols = [col for col in df_train.columns if col.startswith("tfidf_")]
meta_features = [
    "title_len",
    "title_has_digit",
    "description_text_len",
    "is_top20_brand",
    "has_price",
    "is_top9_category_main"
]
features = (
    tfidf_cols +
    meta_features +
    ["brand", "category_main", "price_clean"]
)
cat_features = ["brand", "category_main"]

In [15]:
# Пул для catboost
train_pool = Pool(df_train[features], label=df_train["label"], cat_features=cat_features)

In [16]:
# Инициализируем и обучим модель
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=8,
    l2_leaf_reg=5,
    bootstrap_type="Bayesian",
    eval_metric="AUC",
    random_seed=42,
    verbose=200,
    task_type="GPU"
)

In [17]:
model.fit(train_pool)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 164ms	remaining: 2m 43s
200:	total: 27.2s	remaining: 1m 48s
400:	total: 55.4s	remaining: 1m 22s
600:	total: 1m 24s	remaining: 55.9s
800:	total: 1m 53s	remaining: 28.2s
999:	total: 2m 23s	remaining: 0us


In [18]:
# Предсказываем вероятности для всех товаров из df_meta_tfidf
# Сортируем по вероятности и выбираем top-10 и top-50 для каждого пользователя
user_ids = df_test["user_id"].unique()
predictions = {}
# predictions_pool = {}

In [19]:
for user in tqdm(user_ids, file=sys.stdout):
    df_user = df_meta_tfidf.copy()
    df_user["user_id"] = user

    pool = Pool(df_user[features], cat_features=cat_features)
    df_user["pred"] = model.predict_proba(pool)[:, 1]

    # Оригинальный топ-10 (для грязных метрик)
    predictions[user] = (
        df_user.sort_values("pred", ascending=False)["item_id"]
        .head(10)
        .tolist()
    )

    # # Расширенный пул топ-50
    # predictions_pool[user] = (
    #     df_user.sort_values("pred", ascending=False)["item_id"]
    #     .head(50)
    #     .tolist()
    # )

100%|██████████| 85083/85083 [17:44:04<00:00,  1.33it/s]  


In [20]:
ground_truth = df_test.groupby("user_id")["item_id"].apply(set).to_dict()

In [21]:
# Оцениваем Precision@10 и MAP@10 для модели на TF-IDF по всем товарам
print("Baseline 1 (по всем товарам)")
print("Precision@10:", precision_at_k(predictions, ground_truth, k=10))
print("MAP@10:", map_at_k(predictions, ground_truth, k=10))

Baseline 1 (по всем товарам)
Precision@10: 0.0001
MAP@10: 0.0001


### Matching MLP — нейросетевая модель

Формируется обучающая выборка: конкатенируются векторы пользователя (среднее по вектору его положительных товаров) и вектор текущего товара.

Нейросеть `MatchingMLP` обучается отличать положительные пары от отрицательных.



In [16]:
# Загружаем тренировочную выборку с CLIP-эмбеддингами (текст + изображение)
df_train = pd.read_csv(
    "data/df_train_CLIP_rating_based.csv",
    na_values=[""],  # исключаем "Unknown"
    keep_default_na=False
)

In [17]:
# Выбираем CLIP-эмбеддинги (текст + изображение)
item_vector_cols = [col for col in df_train.columns if col.startswith("clip_text_") or col.startswith("clip_img_")]
user_ids = df_train["user_id"].unique()

In [18]:
# Указываем дополнительные табличные признаки, которые добавим к CLIP-вектору
extra_vec_cols = [
    "title_len",
    "description_text_len",
    "is_top20_brand",
    "has_price",
    "is_top9_category_main",
    "price_clean"
]

In [19]:
# Избавляемся от дубликатов товаров и нормализуем числовые признаки
df_item_extra = df_train.drop_duplicates("item_id")[["item_id"] + extra_vec_cols].copy()

In [20]:
df_item_extra["price_clean"] = df_item_extra["price_clean"].fillna(-1)

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_item_extra[["title_len", "description_text_len", "price_clean"]] = scaler.fit_transform(
    df_item_extra[["title_len", "description_text_len", "price_clean"]]
)

In [22]:
# Переводим в индекс item_id для удобного объединения
df_item_extra = df_item_extra.set_index("item_id")

In [23]:
# Извлекаем CLIP-эмбеддинги и объединяем с табличными признаками
clip_vectors  = df_train.drop_duplicates("item_id")[["item_id"] + item_vector_cols].set_index("item_id")

In [24]:
item_vectors = pd.concat([clip_vectors, df_item_extra], axis=1)

In [25]:
print("extra_vectors shape:", df_item_extra.shape)
print("clip_vectors shape:", clip_vectors.shape)
print("item_vectors shape:", item_vectors.shape)

extra_vectors shape: (67706, 6)
clip_vectors shape: (67706, 200)
item_vectors shape: (67706, 206)


In [26]:
# Формируем вектор интересов пользователя как среднее по позитивным item-векторам
user_vectors = {}

In [27]:
for user_id, group in tqdm(df_train[df_train["label"] == 1].groupby("user_id"), desc="User vector aggregation", file=sys.stdout):
    item_ids = group["item_id"].values
    vectors = item_vectors.loc[item_ids].values
    user_vectors[user_id] = np.mean(vectors, axis=0)

User vector aggregation: 100%|██████████| 551853/551853 [02:51<00:00, 3219.60it/s]


In [28]:
# Строим датасет: конкатенируем векторы пользователя и товара
X = []
y = []

In [29]:
for row in tqdm(df_train.itertuples(), total=len(df_train), desc="Building training pairs", file=sys.stdout):
    item_id = row.item_id
    user_id = row.user_id
    label = row.label

    if user_id not in user_vectors or item_id not in item_vectors.index:
        continue

    user_vec = user_vectors[user_id]
    item_vec = item_vectors.loc[item_id].values

    concat_vec = np.concatenate([user_vec, item_vec])
    X.append(concat_vec)
    y.append(label)

Building training pairs: 100%|██████████| 1635089/1635089 [02:59<00:00, 9090.15it/s]


In [30]:
# Преобразуем списки в массивы для дальнейшей подачи в PyTorch
X = np.array(X)
y = np.array(y)
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (1542533, 412)
y shape: (1542533,)


In [ ]:
# Импортируем PyTorch и определяем, использовать ли GPU
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [33]:
# Определяем Dataset
class MatchingDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [34]:
# Создаём датасет и делим его на обучающую и валидационную части
full_dataset = MatchingDataset(X, y)

In [35]:
train_size = int(0.9 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [36]:
# Оборачиваем датасеты в DataLoader для удобной подачи батчами
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2048)

In [37]:
# Определяем MLP модель
class MatchingMLP(nn.Module):
    def __init__(self, input_dim=400):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [38]:
# Создаём экземпляр модели
model = MatchingMLP(input_dim=X.shape[1]).to(device)

In [39]:
# Используем бинарную кросс-энтропию в качестве функции потерь
# и Adam как оптимизатор с learning rate 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [40]:
from sklearn.metrics import accuracy_score
# Обучаем модель в течение 10 эпох, отслеживая loss и accuracy на валидации
num_epochs = 10

In [41]:
for epoch in range(num_epochs):
    model.train()
    train_losses = []

    for X_batch, y_batch in tqdm(train_loader, desc="Training", file=sys.stdout):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).view(-1, 1)

        preds = model(X_batch)
        loss = criterion(preds, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    # Валидация
    model.eval()
    val_losses = []
    all_preds = []
    all_targets = []

    for X_batch, y_batch in tqdm(val_loader, desc="Validating", file=sys.stdout):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).view(-1, 1)
        with torch.no_grad():
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
        val_losses.append(loss.item())
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(y_batch.cpu().numpy())

    # Бинаризуем предсказания, считаем accuracy и выводим метрики за эпоху
    all_preds_bin = (np.array(all_preds) >= 0.5).astype(int)
    val_acc = accuracy_score(all_targets, all_preds_bin)

    tqdm.write(
        f"Epoch {epoch+1}/{num_epochs} | "
        f"Train Loss: {np.mean(train_losses):.4f} | "
        f"Val Loss: {np.mean(val_losses):.4f} | "
        f"Val Accuracy: {val_acc:.4f}"
    )

Validating: 100%|██████████| 76/76 [00:01<00:00, 48.70it/s]
Epoch 1/10 | Train Loss: 0.2840 | Val Loss: 0.2185 | Val Accuracy: 0.9202
Validating: 100%|██████████| 76/76 [00:01<00:00, 49.98it/s]
Epoch 2/10 | Train Loss: 0.2074 | Val Loss: 0.1671 | Val Accuracy: 0.9403
Validating: 100%|██████████| 76/76 [00:01<00:00, 50.20it/s]
Epoch 3/10 | Train Loss: 0.1558 | Val Loss: 0.1291 | Val Accuracy: 0.9542
Validating: 100%|██████████| 76/76 [00:01<00:00, 55.31it/s]
Epoch 4/10 | Train Loss: 0.1278 | Val Loss: 0.1104 | Val Accuracy: 0.9611
Validating: 100%|██████████| 76/76 [00:01<00:00, 49.98it/s]
Epoch 5/10 | Train Loss: 0.1127 | Val Loss: 0.1015 | Val Accuracy: 0.9643
Validating: 100%|██████████| 76/76 [00:01<00:00, 50.42it/s]
Epoch 6/10 | Train Loss: 0.1036 | Val Loss: 0.0990 | Val Accuracy: 0.9655
Validating: 100%|██████████| 76/76 [00:01<00:00, 54.27it/s]
Epoch 7/10 | Train Loss: 0.0971 | Val Loss: 0.0918 | Val Accuracy: 0.9682
Validating: 100%|██████████| 76/76 [00:01<00:00, 54.78it/s]
Ep

In [42]:
# Проверяем размер и разнообразие тестового сета перед инференсом
print(f"Test shape: {df_test.shape}, unique users: {df_test['user_id'].nunique()}")

Test shape: (138023, 5), unique users: 85083


In [43]:
# Создаём словарь ground truth: реальные товары, 
# с которыми взаимодействовал пользователь
ground_truth = df_test.groupby("user_id")["item_id"].apply(set).to_dict()

In [44]:
# Переводим модель в режим инференса
model.eval()

MatchingMLP(
  (model): Sequential(
    (0): Linear(in_features=412, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=1, bias=True)
    (6): Sigmoid()
  )
)

In [45]:
# Подготавливаем список кандидатов и их эмбеддинги для инференса
candidate_items = item_vectors.index.tolist()
candidate_vectors = item_vectors.values
candidate_vectors.shape

(67706, 206)

In [46]:
# Для каждого пользователя генерируем top-N рекомендаций
# Сортировка кандидатов по вероятности релевантности (score)
top_k = 10
predictions = {}
# predictions_pool = {}

In [47]:
for user_id in tqdm(ground_truth.keys(), desc="Generating top-N", file=sys.stdout):
    if user_id not in user_vectors:
        continue

    user_vec = user_vectors[user_id]
    user_vec_batch = np.tile(user_vec, (candidate_vectors.shape[0], 1))
    concat = np.hstack([user_vec_batch, candidate_vectors])

    with torch.no_grad():
        scores = model(torch.tensor(concat, dtype=torch.float32).to(device)).cpu().numpy().flatten()

    sorted_items = np.array(candidate_items)[np.argsort(scores)[::-1]]
    predictions[user_id] = sorted_items[:10].tolist()
    # predictions_pool[user_id] = sorted_items[:50].tolist()

Generating top-N: 100%|██████████| 85083/85083 [5:53:11<00:00,  4.02it/s]  


In [48]:
# Считаем метрики на всём списке рекомендаций
print("Matching MLP (top-10)")
print("Precision@10:", precision_at_k(predictions, ground_truth))
print("MAP@10:", map_at_k(predictions, ground_truth))

Matching MLP (top-10)
Precision@10: 0.0015
MAP@10: 0.0047


### Сравнение моделей по метрикам качества (rating-based, temporal split)

Все метрики рассчитаны по **всему пулу товаров**, без фильтрации уже просмотренных пользователем товаров.

| Модель                  | Precision@10 | MAP@10  |
|-------------------------|--------------|---------|
| **Baseline 0 (Popular)**| **0.0015**   | 0.002   |
| **Baseline 1 (TF-IDF)** | 0.0001       | 0.0001  |
| **Matching MLP (CLIP)** | **0.0015**       | **0.0047** |

---

### Выводы:

* **Baseline 0 (popular)** показал наивысший **Precision@10**, несмотря на простоту. Это подчёркивает, что в задачах без персонализации популярные товары могут быть сильным ориентиром.
* **Matching MLP** пока не превосходит популярность по точности, но даёт **более высокое MAP@10**, что указывает на **лучшее ранжирование релевантных товаров** в пределах топа.
* **Baseline 1 (TF-IDF + CatBoost)** продемонстрировал крайне низкие результаты — как по точности, так и по позиционному качеству. Это подтверждает ограниченность TF-IDF признаков в условиях rating-based и большого пула товаров.
* Полученные значения задают **честный baseline** для дальнейшего улучшения модели:
  - добавления **user-based признаков**;
  - замены head'а с MLP на **dot-product**;
  - оптимизации признаков и гиперпараметров.

---

Модель работает честно, метрики реалистичны, и мы готовы к улучшениям.
